In [265]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(linewidth=110)
print()


In [18]:
def tsp(cost: list) -> int:
    n = len(cost)
    c = {}
    ss = (1 << n) - 2  # all bits except 0
    q = deque()
    mnc = inf

    print(ss)
    for k in range(1, n):
        ks = ss & ~(1 << k)  # 1 bit less
        c[(ks, k)] = cost[0][k]
        q.append(ks)

    while q:
        print(q)
        ps = q.popleft()
        if ps == 0:
            cmk = min(c.get((ps | (1 << k), k), inf) + cost[k][0] for k in range(1, n))
            mnc = min(mnc, cmk)
        for k, kb in bits(ps):
            assert ps & kb != 0  # noqa: S101
            ks = ps & ~kb  # 1 bit less
            print(ks)
            cmk = min(c.get((ks | b, m), inf) + cost[m][k] for m, b in bits(ks))
            c[(ks, k)] = min(cmk, c.get((ks, k), cmk))
            if cmk < mnc:
                q.append(ks)

    return mnc


In [17]:
def tsp(cost: list) -> int:
    @lru_cache(None)
    def g(s: int, k: int) -> int:
        return min(g(s & ~b, m) + cost[m][k] for m, b in bits(s)) if s else cost[0][k]

    n = len(cost)
    s = (1 << n) - 2
    return s and min(g(s & ~(1 << k), k) + cost[k][0] for k in range(1, n)) or 0


In [102]:
def tsp(cost: list) -> int:
    n = len(cost)
    full = (1 << n) - 1
    memo = [[-1] * (1 << n) for _ in range(n)]

    def f(i: int, mask: int) -> int:
        if mask == full:
            return cost[i][0]
        if memo[i][mask] >= 0:
            return memo[i][mask]

        res = 10**9  # result of this sub-problem
        for j in range(n):
            if mask & (1 << j) == 0:
                res = min(res, cost[i][j] + f(j, mask | (1 << j)))
        memo[i][mask] = res
        return res

    res = f(0, 1)
    print(memo)
    return res


In [53]:
def tsp(cost: list) -> int:
    n = len(cost)
    base = (1 << n) - 1

    @lru_cache(None)
    def g(s: int, k: int) -> int:
        if s == base:
            return cost[k][0]
        r = 10**10
        for i in range(n):
            if s & (1 << i) == 0:
                r = min(r, cost[k][i] + g(s | (1 << i), i))
        return r

    return g(1, 0)


In [272]:
def tsp(cost: list) -> int:
    @lru_cache(None)
    def g(s: int, k: int) -> int:
        return min(cost[k][i] + g(s & ~b, i) for i, b in bits.enum(s)) if s else cost[k][0]

    return g((1 << len(cost)) - 2, 0)


In [111]:
def tsp(cost: list) -> int:
    n = len(cost)
    m = [[-1] * (1 << n) for _ in range(n)]

    def g(s: int, k: int) -> int:
        if not s:
            return cost[k][0]
        if m[k][s] < 0:
            m[k][s] = min(cost[k][i] + g(s & ~b, i) for i, b in bits(s))
        return m[k][s]

    r = g((1 << len(cost)) - 2, 0)
    print(m)
    return r


In [126]:
def tsp(cost: list) -> int:
    n = len(cost)
    m = [[-1] * (1 << n) for _ in range(n)]
    mask = (1 << n) - 1
    q = []
    for k in range(n):
        c = m[k][0] = cost[k][0]
        heappush(q, (c, 1 << k))

    while q:
        s = q.popleft()
        for k, b in bits(mask & ~s):
            if m[k][s] < 0:
                m[k][s] = min(cost[k][i] + m[i][s & ~bit] for i, bit in bits(s))
                q.append(s | b)
    # print(m)

    return m[0][mask - 1]


In [212]:
def tsp(cost: list) -> int:
    n = len(cost)
    # path = [0] * n

    min_out = [min(cost[i][j] for j in range(n) if j != i) for i in range(n)]
    # min_in = [min(cost[i][j] for i in range(n) if j != i) for j in range(n)]

    mn = inf

    def r(lb: int, u: int, c: int, level: int, unv: int) -> None:
        nonlocal mn
        if level == 0:
            mn = min(mn, c + cost[u][0])
            return
        for i, b in bits(unv):
            ic = c + cost[u][i]
            ilb = lb - min_out[i]
            if ic + ilb <= mn:
                # path[-level] = i
                r(ilb, i, ic, level - 1, unv & ~b)

    lb = sum(min_out[1:])
    r(lb, 0, 0, n - 1, (1 << n) - 2)
    return mn


In [268]:
def tsp(cost: list[list[int]]) -> int:
    """Return the exact minimum path cost of the traveling salesman problem."""
    n = len(cost)
    if n < 2:
        return 0

    # Used for the lower bound estimation.
    min_out = [min(cost[i][j] for j in range(n) if j != i) for i in range(n)]

    # The resulting minimum path cost.
    mn: int = sum(max(cost[i][j] for j in range(n)) for i in range(n))
    # Stack containing traversal states.
    # lower-bound, cost, unvisited, node-index
    s: list[tuple] = [(sum(min_out), 0, (1 << n) - 2, 0)]

    while s:
        h, c, unvisited, u = s.pop()
        if unvisited == 0:
            mn = min(mn, c + cost[u][0])
            continue
        # Cut by the best solution as early as possible.
        if h >= mn:
            continue
        h -= min_out[u]
        # Iterate over the unvisited node indexes.
        for i, b in bits.enum(unvisited):
            ih = h + cost[u][i]
            # Cut by the best solution found so far.
            if ih <= mn:
                s.append((ih, c + cost[u][i], unvisited & ~b, i))

    return mn


In [273]:
g = grid.parse(
    """\
107 816 247 307 92 417 423 55 365 876 435 456 299 101 577 324 10 335 978 130
94 146 382 92 570 256 895 664 574 853 571 507 301 493 250 952 584 144 319 61
365 916 585 259 660 605 759 975 189 698 449 670 264 140 359 372 281 698 157 772
839 860 603 100 651 676 711 572 505 120 499 423 904 389 635 431 710 234 260 866
93 254 382 986 18 840 676 346 94 706 457 937 305 325 798 299 738 84 810 996
539 137 723 644 620 712 389 126 490 450 594 717 646 169 856 273 495 279 57 848
909 307 516 917 945 78 559 882 704 155 670 715 117 335 777 971 385 367 974 325
161 969 7 347 475 973 158 601 703 693 460 307 994 174 193 519 218 352 449 227
962 602 600 942 197 912 690 224 578 266 888 145 366 564 876 978 920 356 219 577
441 808 949 345 914 614 407 162 239 830 76 416 28 282 137 273 856 578 643 957
394 794 246 421 237 23 522 68 457 105 684 572 603 4 722 129 587 663 84 667
201 295 715 818 883 274 901 816 400 515 922 104 794 844 577 73 737 692 229 660
361 454 720 921 3 142 61 151 297 304 791 283 291 820 158 74 504 799 125 551
211 721 293 143 690 557 984 726 212 834 989 236 469 344 822 844 462 251 494 868
874 204 899 266 188 63 899 941 841 968 320 104 206 68 494 918 811 983 529 767
595 677 681 160 336 341 837 145 404 991 504 440 16 335 254 851 1000 491 769 98
333 588 694 477 675 595 858 149 972 677 761 468 970 54 991 374 68 832 783 137
197 225 379 451 654 298 138 809 133 854 105 732 614 864 525 55 39 144 431 504
352 39 151 380 551 218 708 865 241 898 70 322 5 110 102 369 269 89 601 704
115 519 166 58 81 231 71 129 612 143 228 540 697 590 968 624 435 772 215 556"""
)
tsp(g)


1708

In [253]:
g = grid.parse(
    """\
0 9 6 8
4 0 3 1
9 8 0 6
4 7 5 0"""
)
tsp(g)


19

In [254]:
tsp([[0]])


0

In [255]:
g = grid.parse(
    """\
0 2 7 7
1 0 9 9
1 8 0 9
5 2 10 0"""
)
tsp(g)


19

In [240]:
g = grid.parse(
    """\
0 2167 3150 2813 8453 9303 8998 1918
5018 0 4676 8015 1595 6768 6350 5244
7573 9787 0 3790 547 7036 9899 8995
1117 5451 7862 0 4295 8814 5240 1755
3228 9558 8463 711 0 8959 5713 8069
9601 8133 3995 1343 3068 0 347 6450
7294 6795 164 2589 4651 5219 0 9887
6554 4997 2876 7940 2550 7778 1101 0"""
)
tsp(g)


16569

In [221]:
g = grid.parse(
    """\
0 4430 6977 2797 9384 5713 7860 8499
8061 0 8306 7416 545 3111 3344 6000
1891 7812 0 5617 9606 8926 790 1251
961 6814 6060 0 6914 4297 3065 6399
8986 9587 1126 1090 0 5199 594 9271
5846 15 8138 4425 7613 0 4242 3207
4997 5408 5716 5024 1763 7331 0 5057
4229 2391 6263 7616 8667 2987 1953 0"""
)
tsp(g)


16588

In [222]:
tsp([[0, 111], [112, 0]])


223

In [223]:
tsp([[0, 1000, 5000], [5000, 0, 1000], [1000, 5000, 0]])


3000

In [224]:
tsp([[0]])


0

In [225]:
tsp([[0, 111, 555], [112, 0, 555], [113, 222, 0]])


779